In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.linear_model import LinearRegression

As this income has little datapoints, use simple linear model

In [2]:
# Load the dataset
file_path = '../../data/raw/Past_income_population_preprocessed/income_2020_filtered.csv'
income_data = pd.read_csv(file_path)

# Replace 'np' with NaN and remove commas from the income values
income_data.replace('np', np.nan, inplace=True)
income_data = income_data.apply(lambda x: x.str.replace(',', '') if x.dtype == "object" else x)
income_data = income_data.apply(pd.to_numeric, errors='ignore')  # Convert to numeric

# Fill missing values (forward fill, then backward fill if needed)
income_data = income_data.fillna(method='ffill').fillna(method='bfill')

# Prepare for Linear Regression forecasting (we will forecast for each SA2 Code separately)
forecast_results = []

# Loop through each SA2 Code to forecast 2025-2027 income
for idx, row in income_data.iterrows():
    sa2_code = row['SA2 Code']
    
    # Extract income data for the specific SA2 Code (2016-2020)
    time_series_data = row[['2016 income', '2017 income', '2018 income', '2019 income', '2020 income']].values
    
    # Prepare the years (2016 to 2020) as the independent variable
    years = np.array([2016, 2017, 2018, 2019, 2020]).reshape(-1, 1)
    
    # Fit Linear Regression model
    linear_model = LinearRegression()
    linear_model.fit(years, time_series_data)

    # Forecast income for 2024, 2025, 2026, and 2027 using Linear Regression
    future_years = np.array([2024, 2025, 2026, 2027]).reshape(-1, 1)
    forecast = linear_model.predict(future_years)

    # Append results (SA2 code and forecasted values)
    forecast_results.append([sa2_code, forecast[0], forecast[1], forecast[2], forecast[3]])

# Save the forecast to a new CSV file
forecast_df = pd.DataFrame(forecast_results, columns=['SA2 Code', '2024 income', '2025 income', '2026 income', '2027 income'])
output_file_path = '../../data/raw/Past_income_population_preprocessed/income_forecast_2024_2027.csv'
forecast_df.to_csv(output_file_path, index=False)

print(f"Forecast saved to {output_file_path}")


/tmp/ipykernel_227202/2103550411.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  income_data = income_data.apply(pd.to_numeric, errors='ignore')  # Convert to numeric
/tmp/ipykernel_227202/2103550411.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  income_data = income_data.fillna(method='ffill').fillna(method='bfill')


Forecast saved to ../../data/raw/Past_income_population_preprocessed/income_forecast_2024_2027.csv
